In [1]:
import sys, os
sys.path.insert(0, 'D:\Senior Year Northeastern University\DS Capstone\ImageOutpainting\model')
sys.argv[1] = "pix2pix"
#print(sys.path)

import random
random.seed(0)

import torch
from PIL import ImageColor, Image
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

from train import main
from masking.mask_transforms import Datum, ApplyMaskTransform, RandomRightTransform, MaskedAreaTransform
from masking.mask_gen import RightStripeMaskGenerator, TopStripeMaskGenerator
from config import config

ImportError: cannot import name 'main' from 'train' (D:\Senior Year Northeastern University\DS Capstone\ImageOutpainting\model\train.py)

In [ ]:
gen = main()

In [ ]:
def shift_image(self, img):
    if not self.config.recur_bool:
        return img
    elif self.config.recur_direction == "top":
        return img[:,
                  self.config.stripe_width::,
                  self.config.stripe_width:,
                  self.config.stripe_width:]
    elif self.config.recur_direction == "bottom":
        return img[:,
                  :self.config.pic_height-self.config.stripe_width,
                  :self.config.pic_height-self.config.stripe_width,
                  :self.config.pic_height-self.config.stripe_width]
    elif self.config.recur_direction == "left":
        return img[:,
                  self.config.stripe_width:,
                  self.config.stripe_width:,
                  self.config.stripe_width:]
    elif self.config.recur_direction == "right":
        return img[:,
                  :self.config.pic_width-self.config.stripe_width,
                  :self.config.pic_width-self.config.stripe_width,
                  :self.config.pic_width-self.config.stripe_width]
    else:
        raise ValueError(f"Cannot shift image in direction {self.config.recur_direction}")

In [ ]:
class Utils:
    def show(self, img):
        toPIL = transforms.ToPILImage()
        img = toPIL(img[:, :3, :, :][0])
        plt.imshow(img)
u = Utils()

In [ ]:
class ImageLoader:
    def __init__(self, filepaths):
        self.filepaths = filepaths
    
    def __iter__(self):
        for fp in self.filepaths:
            to_yield = Image.open(fp), None
            yield to_yield
            
class LeftImageLoader:
    def __init__(self, filepaths):
        self.filepaths = filepaths
    
    def __iter__(self):
        for fp in self.filepaths:
            to_yield = Image.open(fp), None
            yield to_yield

In [ ]:
class RecurDataset(torch.utils.data.IterableDataset):
    
    def __init__(self):
        super(OutsideRightDataset).__init__()
        self.dataset = self.load_dataset()
                
    def __iter__(self):
        for x, _ in iter(self.dataset):
            both = transforms.Compose([
                transforms.Resize((config.pic_height, config.pic_width)),
                transforms.ToTensor(),
                RandomRightTransform(config, right_chance = 1.1)
            ])

            xonly = ApplyMaskTransform(config, extra_dim=(not config.should_collapse))
            yonly = MaskedAreaTransform(config)
            p = both(x)
            yield xonly(p), yonly(p)
            
    def __len__(self):
        return len(self.train_dataset)
    
    def get_pic_filepaths(self):
        for pic in os.listdir("model\dataset\places365_standard\train\boardwalk"):
            yield os.path.join("model\dataset\places365_standard\train\boardwalk", pic)
                
    def load_dataset(self):
        filepaths = list(self.get_pic_filepaths())
        return RightImageLoader(filepaths)

In [ ]:
recur_loader = RecurDataset().dataset


In [ ]:
both = transforms.Compose([
                transforms.Resize((config.pic_height, config.pic_width)),
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                RandomRightTransform(config, right_chance = -0.1)
            ])
xonly = ApplyMaskTransform(config, extra_dim=(not config.should_collapse))
yonly = MaskedAreaTransform(config)

for i, (X, y) in enumerate(recur_loader):
    p = both(X)
    x_, y_ = xonly(p), yonly(p)
    x_, y_ = torch.reshape(x_, (1, x_.shape[0], x_.shape[1], x_.shape[2])), torch.reshape(y_, (1, y_.shape[0], y_.shape[1], y_.shape[2]))
    x_ = x_.to(torch.device('cuda'))
    print(x_.device)
    break
    
u.show(x_)
print(x_.shape)

In [ ]:
u.show(gen(x_))